In [1]:
import json
import spacy
from spacy.tokens import Doc
from spacy.lang.en import English
from spacy.matcher import Matcher
from spacy import displacy

# Name Entity Recognition Task
## Data Transformation and EDA

In [2]:
with open('sentences_data.json', 'r') as json_file:
    data = json.load(json_file)
data = json.loads(data)

In [3]:
nlp = spacy.load('en_core_web_md')

In [4]:
# Register the Doc extension "title" (default None)
Doc.set_extension("title", default=None)

# Register the Doc extension "gddid" (default None)
Doc.set_extension("gddid", default=None)

# Register the Doc extension "sentid" (default None)
Doc.set_extension("sentid", default=None)

In [5]:
for doc, context in nlp.pipe(data, as_tuples=True):
    # Set the doc._.book and doc._.author attributes from the context
    doc._.title = str(context["title"])
    doc._.gddid = str(context["gddid"])
    doc._.sentid = int(context["sentid"])

    # Print the text and custom attribute data
    # print(f"{doc._.sentid} - {doc.text}\n — 'TITLE': {doc._.title}' GDDID': {doc._.gddid}\n")
    #for ent in doc.ents:
    # Print the entity text and label
    #    if ent.label_ == "LOC": # change to location
    #        print(ent.text, ent.label_)

In [11]:
# DD Paterns
#dd_pattern = [{"LOWER": {"REGEX": "([-]?\d{1,3}\.\d{1,}[,]?[NESWnesw])"}}, {"TEXT":","}, {"LOWER": {"REGEX": "([-]?\d{1,3}\.\d{1,}[,]?[NESWnesw])"}}]
dd_pattern2 = [[{'LIKE_NUM': True}, {"TEXT": {"REGEX" : "[°|o|◦|′|\'|`|\"]"}}, {"LOWER": {"REGEX": "^[nesw]$"}}, {"TEXT":","},{'LIKE_NUM': True}, {"TEXT": {"REGEX" : "[°|′|o|\'|,]"}}, {"LOWER": {"REGEX": "^[nesw]$"}}]]

# DMS Patterns
dms_pattern = [[{'LIKE_NUM': True}, {"TEXT": {"REGEX" : "[°|o|◦|′|\'|`|\"]"}}, {'LIKE_NUM': True}, {"TEXT": {"REGEX" : "[°|o|◦|′|\'|`|\"]"}}, {'LIKE_NUM': True}, {"TEXT": {"REGEX" : "[°|o|◦|′|\'|`|\"]"}}]]
dms_pattern2 = [[{'LIKE_NUM': True}, {"TEXT": {"REGEX" : "[°|o|◦|′|\'|`|\"]"}}, {'LIKE_NUM': True}, {"TEXT": {"REGEX" : "[°|o|◦|′|\'|`|\"]"}}, {"LOWER": {"REGEX": "([-]?\d{1,3}?[nesw])"}}]]

# textual patterns
text_pattern = [[{'LIKE_NUM': True}, {"TEXT": {"REGEX" : "[°|o|◦|′|\'|`|\"]"}}, {"LOWER": {"REGEX": "^[nesw]$"}}, {"TEXT": ","}, {'LIKE_NUM': True}, {"LOWER": "minutes"}, {"TEXT": ","}, {"LOWER":"and"},{'LIKE_NUM': True}, {"LOWER": "seconds"}]]


In [12]:
# Making sure that coordinates do not overlap
strategies = [dd_pattern2, dms_pattern, dms_pattern2, text_pattern]
entities = []
for strategy in strategies:
    matcher = Matcher(nlp.vocab)
    matcher.add("COORDS", strategy)
    for doc, context in nlp.pipe(data, as_tuples=True):
        matches = matcher(doc)
        # Match on the doc and create a list of matched spans
        if len(matches) > 0:
            spans = [doc[start:end] for match_id, start, end in matcher(doc)]
            # Initialize auxiliary lists

            # Get (start character, end character, label) tuples of matches
            coord_entities = [(span.start_char, span.end_char, "COORDS") for span in spans]      
            new_list = []

            for idx, item in enumerate(coord_entities):
                #print("element", item)
                if new_list == []:
                    new_list.append(item)
                else:
                    aux_list = []
                    for idx2, item2 in enumerate(new_list):
                        if item[0] in range(item2[0], item2[1]+1):
                            aux_list.append(item)
                        if item[1] in range(item2[0], item2[1]+1):
                            aux_list.append(item)

                    if item not in aux_list:
                        new_list.append(item)
            entities += new_list
            print(entities)
            
        #sym_diff_list = list(set(intersection_list_a).symmetric_difference(set(coord_entities)))

len(entities)

[(178, 196, 'COORDS')]
[(178, 196, 'COORDS'), (695, 709, 'COORDS')]
[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS')]
[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS')]
[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS')]
[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS'), (84, 98, 'COORDS')]
[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS'), (84, 98, 'COORDS'), (41, 55, 'COORDS')]
[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS'), (84, 98, 'COORDS'), (41, 55, 'COORDS'), (45, 63, 'COORDS')]
[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS'), (84, 98, 'COORDS'), (41, 55, 'COORDS'), (45, 63, 'COORDS'), (112, 132, 'COORDS'), (157, 177, 'COORDS')]
[(178, 196, 'COORDS')

[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS'), (84, 98, 'COORDS'), (41, 55, 'COORDS'), (45, 63, 'COORDS'), (112, 132, 'COORDS'), (157, 177, 'COORDS'), (750, 773, 'COORDS'), (2090, 2110, 'COORDS'), (2617, 2639, 'COORDS'), (123, 140, 'COORDS'), (191, 214, 'COORDS'), (197, 211, 'COORDS'), (341, 364, 'COORDS'), (748, 773, 'COORDS'), (115, 127, 'COORDS'), (146, 162, 'COORDS'), (149, 165, 'COORDS'), (14, 28, 'COORDS'), (0, 16, 'COORDS'), (0, 18, 'COORDS'), (62, 82, 'COORDS'), (0, 17, 'COORDS'), (30, 53, 'COORDS'), (1045, 1062, 'COORDS'), (1063, 1083, 'COORDS'), (1084, 1101, 'COORDS'), (1102, 1122, 'COORDS'), (1123, 1140, 'COORDS'), (1141, 1161, 'COORDS'), (1162, 1179, 'COORDS'), (1180, 1200, 'COORDS'), (1201, 1218, 'COORDS'), (1219, 1239, 'COORDS'), (1240, 1257, 'COORDS'), (1258, 1278, 'COORDS'), (1279, 1296, 'COORDS'), (1297, 1317, 'COORDS'), (1318, 1335, 'COORDS'), (1336, 1356, 'COORDS'), (1357, 1374, 'COORDS'), (1375, 1395, 'COOR

[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS'), (84, 98, 'COORDS'), (41, 55, 'COORDS'), (45, 63, 'COORDS'), (112, 132, 'COORDS'), (157, 177, 'COORDS'), (750, 773, 'COORDS'), (2090, 2110, 'COORDS'), (2617, 2639, 'COORDS'), (123, 140, 'COORDS'), (191, 214, 'COORDS'), (197, 211, 'COORDS'), (341, 364, 'COORDS'), (748, 773, 'COORDS'), (115, 127, 'COORDS'), (146, 162, 'COORDS'), (149, 165, 'COORDS'), (14, 28, 'COORDS'), (0, 16, 'COORDS'), (0, 18, 'COORDS'), (62, 82, 'COORDS'), (0, 17, 'COORDS'), (30, 53, 'COORDS'), (1045, 1062, 'COORDS'), (1063, 1083, 'COORDS'), (1084, 1101, 'COORDS'), (1102, 1122, 'COORDS'), (1123, 1140, 'COORDS'), (1141, 1161, 'COORDS'), (1162, 1179, 'COORDS'), (1180, 1200, 'COORDS'), (1201, 1218, 'COORDS'), (1219, 1239, 'COORDS'), (1240, 1257, 'COORDS'), (1258, 1278, 'COORDS'), (1279, 1296, 'COORDS'), (1297, 1317, 'COORDS'), (1318, 1335, 'COORDS'), (1336, 1356, 'COORDS'), (1357, 1374, 'COORDS'), (1375, 1395, 'COOR

[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS'), (84, 98, 'COORDS'), (41, 55, 'COORDS'), (45, 63, 'COORDS'), (112, 132, 'COORDS'), (157, 177, 'COORDS'), (750, 773, 'COORDS'), (2090, 2110, 'COORDS'), (2617, 2639, 'COORDS'), (123, 140, 'COORDS'), (191, 214, 'COORDS'), (197, 211, 'COORDS'), (341, 364, 'COORDS'), (748, 773, 'COORDS'), (115, 127, 'COORDS'), (146, 162, 'COORDS'), (149, 165, 'COORDS'), (14, 28, 'COORDS'), (0, 16, 'COORDS'), (0, 18, 'COORDS'), (62, 82, 'COORDS'), (0, 17, 'COORDS'), (30, 53, 'COORDS'), (1045, 1062, 'COORDS'), (1063, 1083, 'COORDS'), (1084, 1101, 'COORDS'), (1102, 1122, 'COORDS'), (1123, 1140, 'COORDS'), (1141, 1161, 'COORDS'), (1162, 1179, 'COORDS'), (1180, 1200, 'COORDS'), (1201, 1218, 'COORDS'), (1219, 1239, 'COORDS'), (1240, 1257, 'COORDS'), (1258, 1278, 'COORDS'), (1279, 1296, 'COORDS'), (1297, 1317, 'COORDS'), (1318, 1335, 'COORDS'), (1336, 1356, 'COORDS'), (1357, 1374, 'COORDS'), (1375, 1395, 'COOR

[(178, 196, 'COORDS'), (695, 709, 'COORDS'), (79, 93, 'COORDS'), (120, 135, 'COORDS'), (55, 75, 'COORDS'), (84, 98, 'COORDS'), (41, 55, 'COORDS'), (45, 63, 'COORDS'), (112, 132, 'COORDS'), (157, 177, 'COORDS'), (750, 773, 'COORDS'), (2090, 2110, 'COORDS'), (2617, 2639, 'COORDS'), (123, 140, 'COORDS'), (191, 214, 'COORDS'), (197, 211, 'COORDS'), (341, 364, 'COORDS'), (748, 773, 'COORDS'), (115, 127, 'COORDS'), (146, 162, 'COORDS'), (149, 165, 'COORDS'), (14, 28, 'COORDS'), (0, 16, 'COORDS'), (0, 18, 'COORDS'), (62, 82, 'COORDS'), (0, 17, 'COORDS'), (30, 53, 'COORDS'), (1045, 1062, 'COORDS'), (1063, 1083, 'COORDS'), (1084, 1101, 'COORDS'), (1102, 1122, 'COORDS'), (1123, 1140, 'COORDS'), (1141, 1161, 'COORDS'), (1162, 1179, 'COORDS'), (1180, 1200, 'COORDS'), (1201, 1218, 'COORDS'), (1219, 1239, 'COORDS'), (1240, 1257, 'COORDS'), (1258, 1278, 'COORDS'), (1279, 1296, 'COORDS'), (1297, 1317, 'COORDS'), (1318, 1335, 'COORDS'), (1336, 1356, 'COORDS'), (1357, 1374, 'COORDS'), (1375, 1395, 'COOR

117

In [14]:
TRAINING_DATA = []
TESTING_DATA = []

strategies = {"dd_pattern2":dd_pattern2, "dms_pattern": dms_pattern, "dms_pattern2":dms_pattern2}

#strategies = [dd_pattern, dd_pattern2, dd_pattern3, dms_pattern, dms_pattern2, text_pattern]
entities = []
for strategy, pattern in strategies.items():
    print(strategy)
    matcher = Matcher(nlp.vocab)
    matcher.add("COORDS", pattern)

    for doc, context in nlp.pipe(data, as_tuples=True):
        matches = matcher(doc)
        # Match on the doc and create a list of matched spans
        if len(matches) > 0:
            spans = [doc[start:end] for match_id, start, end in matcher(doc)]
            print(spans)
            # Initialize auxiliary lists
            intersection_list = []
            # Get (start character, end character, label) tuples of matches
            coord_entities = [(span.start_char, span.end_char, "COORDS") for span in spans]
            

            # make sure coords dont overlap
            new_list = []
            for idx, item in enumerate(coord_entities):
                #print("element", item)
                if new_list == []:
                    new_list.append(item)
                else:
                    aux_list = []
                    for idx2, item2 in enumerate(new_list):
                        if item[0] in range(item2[0], item2[1]+1):
                            aux_list.append(item)
                        if item[1] in range(item2[0], item2[1]+1):
                            aux_list.append(item)

                    if item not in aux_list:
                        new_list.append(item)

            coords_copy = new_list.copy() # aux list
    
            # OLD entities
            old_entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents if ent.label_ == "LOC" or ent.label_ == "GPE"]
            # All entities
            for old_entity in old_entities:
                for coords in coords_copy:
                    if old_entity[0] in range(coords[0], coords[1]):
                        intersection_list.append(old_entity)
                    if old_entity[1] in range(coords[0], coords[1]):
                        intersection_list.append(old_entity)

            sym_diff_list = list(set(intersection_list).symmetric_difference(set(old_entities)))

            entities = sym_diff_list + coord_entities
            #entities3 = entities + entities2
            # Format the matches as a (doc.text, entities) tuple
            training_example = (doc.text, {"entities": entities})
            # Append the example to the training data
            TRAINING_DATA.append(training_example)
        
        else:
            old_entities = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents if ent.label_ == "LOC" or ent.label_ == "GPE"]
            testing_example = (doc.text, {"entities": entities})
            TESTING_DATA.append(testing_example)


dd_pattern2
[5227 ' N, 5626 ' W]
[34 ° S, 41 ° S]
[54 ° S, 41 ° S]
[4 ' S, 6907 ' W]
[1617 ° S, 68.570 ° W]
[20 ° S, 68 ° W]
[20 ° S, 68 ° W]
[3748 ° N, 8067 ° W]
[42.50 ° S, 71.40 ° W, 42.20 ° S, 71.17 ° W]
dms_pattern
[8 Topsail 9 Banook 44 ◦, 23 Long 24 Loon 44 ◦, 29 Power 30 Rocky 44 ◦]
[114 ° 110 ° 106 °, 110 ° 106 ° 102 °, 106 ° 102 ° 98 °]
[1 Tooth 1 Tooth 1 Tooth, 1 Tooth 1 Tooth 2 Postcranial]
[35 ' 30 ' 25 ']
[0 Group 1 Group 2 Group]
[10 Group 1 Group 2 Group3]
[4 ° 30 ' 5 °, 30 ' 5 ° 10 `, 73 ° 4574 ° 25 `]
[60 ° 3063 ° 30 `]
[74 ° 72 ° 70 °, 72 ° 70 ° 68 °, 70 ° 68 ° 66 °]
[35 ° 15 ′ 5135 °, 15 ′ 5135 ° 15 ′]
[106 ° 18 ′ 28106 °, 18 ′ 28106 ° 18 ′]
[1 Soil 2 Soil 3 Soil, 2 Soil 3 Soil 4 Soil]
[0.5 ' 0.6 ` 0.7 ']
[150 Kilometers 92 ° 0 ']
[30 ° 56.20 ′ 31 °, 56.20 ′ 31 ° 08.54 ′, 31 ° 08.54 ′ 31 °, 08.54 ′ 31 ° 08.54 ′, 31 ° 08.54 ′ 31 °, 08.54 ′ 31 ° 03.50 ′, 31 ° 03.50 ′ 31 °, 03.50 ′ 31 ° 08.54 ′, 31 ° 08.54 ′ 31 °, 08.54 ′ 31 ° 08.43 ′, 31 ° 08.43 ′ 31 °, 08.43 ′ 31 ° 0

In [15]:
len(TRAINING_DATA)

46

In [16]:
with open('testing_data.json', 'w') as f:
    json.dump(TESTING_DATA, f)

In [ ]:
#print(*TRAINING_DATA, sep="\n")

In [17]:
TRAINING_DATA[1]

('Submit your article to this journal Article views : 23 View related articles Citing articles : 107 View citing articles Full Terms & Conditions of access and use can be found at http://www.tandfonline.com/action/journalInformation?journalCode=tpal20 Download byLSB- UW-Madison (GeoDeepDive) -RSB- Date : 19 March 2017, At : 06:12 LATE AND POSTGLACIAL VEGETATIONAL AND PALEOCLIMATIC CHANGES IN SUBANTARCTIC, TEMPERATE, AND ARID ENVIRONMENTS IN ARGENTINA VERA MARKGRAF Institute of Arctic and Alpine Research Universityof Colorado Boulder, Colorado 80309 Abstract A paleoclimatic chronology for southern South America is suggested by comparison of dated pollen records from Argentina (lat 32 ° to 34 ° S, 41 ° S and 51 ° to 54 ° S) .',
 {'entities': [(602, 615, 'LOC'),
   (672, 681, 'GPE'),
   (529, 536, 'GPE'),
   (443, 452, 'GPE'),
   (520, 528, 'GPE'),
   (538, 546, 'GPE'),
   (695, 709, 'COORDS')]})

### Right indexes

In [18]:
select_idx = [1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 16, 21, 22, 28, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]

In [19]:
import pandas as pd
a_series = pd.Series(TRAINING_DATA)
accessed_series = a_series[select_idx]
accessed_list = list(accessed_series)

In [21]:
accessed_list[1]

('The next following climatic change occurred at 5 000 yr B.Pwhen at lat 51 ° to 54 ° S, 41 ° S and 31 ° to 34 ° S precipitation increased, especially the winter rain component at lat 34 ° S .',
 {'entities': [(79, 93, 'COORDS')]})